# Your First RAQA Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAQA application. 

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

Let's look at a rather complicated looking visual representation of a basic RAQA application.

<img src="https://i.imgur.com/PvlaIUO.png" />

### Imports and Utility 

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
!pip install -q -U numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

In [11]:
from aimakerspace.text_utils import CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [12]:
import nest_asyncio
nest_asyncio.apply()

# Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with. 

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format. 

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [5]:
# text_loader = TextFileLoader("data/KingLear.txt")
# documents = text_loader.load_documents()
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("2017-mazda3-owners-manual.pdf")
documents = loader.load_and_split() # creates 1 documents per page of the source pdf document

In [13]:
documents = [document.page_content for document in documents]

AttributeError: 'str' object has no attribute 'page_content'

### Splitting Text Into Chunks

As we can see, there is one document - and it's the entire text of Frakenstein

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM. 

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length. 

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following: 
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

Let's take a peek visually at what we're doing here - and why it might be useful:

<img src="https://i.imgur.com/rtM6Ci6.png" />

As you can see (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [7]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

1391

Let's take a look at some of the documents we've managed to split.

In [10]:
print(split_documents[98:101])

['2–29\nEssential Safety Equipment\nChild-Restraint\n Anchor Bracket\nAnchor brackets for securing child-\nrestraint systems are equipped in the \nvehicle. Locate each anchor position using \nthe illustration.\nTo install a child-restraint system, remove \nthe head restraint. Always follow the \ninstruction manual accompanying the \nchild-restraint system.\nAnchor bracket location\nUse the indicated anchor bracket locations \nwhen installing a child-restraint system \nequipped with a tether.\nAnchor bracketFor right\nFor leftFor center(4-Door)\n(5-Door)\nFor left\nFor center For rightSome models.WARNING\nAlways attach the tether strap to the \ncorrect tether anchor position:\nAttaching the tether strap to the \nincorrect tether anchor position is \ndangerous. In a collision, the tether \nstrap could come off and loosen the \nchild-restraint system. If the child-\nrestraint system moves it could result \nin death or injury to the child.\nAlways remove the head restraint and \nset the te

### Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format. 

Loosely, this means turning the text into numbers. 

There are plenty of resources that talk about this process in great detail - I'll leave this [blog](https://txt.cohere.com/sentence-word-embeddings/) from Cohere.AI as a resource if you want to deep dive a bit. 

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

While this is all baked into 1 call - let's look at some of the code that powers this process:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model. 

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓QUESTION:

Though 1536 is the default dimension of `text-embedding-3-small`, you can pass a dimension parameter and shorten the vector - what method does OpenAI use to achieve this shortening? Matryoshka Representation Learning https://arxiv.org/abs/2205.13147

In [14]:
from dotenv import load_dotenv
load_dotenv()
import os

In [9]:
# import os
# import openai
# from getpass import getpass

# openai.api_key = getpass("OpenAI API Key: ")
# os.environ["OPENAI_API_KEY"] = openai.api_key

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [15]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓QUESTION:

What advantage does an async implementation provide for us?

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus. 

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [16]:
# vector_db.search_by_text("Your servant Kent. Where is your servant Caius?", k=3)
vector_db.search_by_text("Where is the tire pressure label on my car?", k=3)

[("8–30\nCustomer Information and Reporting Safety Defects\nTire Information (U.S.A.)\nLocation of the Tire Label (Placard)\nYou will find the tire label containing tire inflation pressure by tire size and other important \ninformation on the driver's side B-pillar or on the edge of the driver's door frame.\nSAMPLE\n Recommended Tire Inflation Pressure\nOn the tire label you will find the recommended tire inflation pressure in both kPa and \npsi for the tires installed as original equipment on the vehicle. It is very important that the \ninflation pressure of the tires on your vehicle is maintained at the recommended pressure. \nYou should check the tire pressure regularly to insure that the proper inflation pressure is \nmaintained.\nRefer to Tires on page 9-10.\nNOTE\nTire pressures listed on the vehicle placard or tire information label indicate the \nrecommended cold tire inflation pressure, measured when the tires are cold, after the vehicle \nhas been parked for at least 3 hours.

# Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work. 

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)): 

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this: 

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts. 

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓QUESTION:

How could we ensure our LLM always responded the same way to our inputs? Set the temperate parameter to zero

### Creating and Prompting OpenAI's `gpt-3.5-turbo`!

Let's tie all these together and use it to prompt `gpt-3.5-turbo`!

In [19]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]

response = chat_openai.run(messages)

In [13]:
print(response)

The best way to write a loop depends on the specific requirements of your code, but generally, I recommend following these principles:

1. Use the right loop type: Choose between a for loop and a while loop based on the situation. Use a for loop when you know the number of iterations in advance, and use a while loop when the termination condition is based on a runtime expression.

2. Keep it simple and readable: Avoid overly complex or nested loops that can be difficult to understand. Make sure the loop body is concise and clearly conveys its purpose.

3. Initialize loop variables properly: When using a for loop, make sure to properly initialize the iterator variable. When using a while loop, ensure that the loop control variable is set correctly to avoid infinite loops.

4. Ensure proper termination: Whether using a for loop or a while loop, make sure that the termination condition is correctly defined to prevent the loop from running indefinitely.

5. Handle exceptions: If there are 

### Retrieval Augmented Question Answering Prompt

Now we can create a RAQA prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [20]:
RAQA_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".

Make sure you first confirm that the context is relevant to the user's query.
It you decide the response is not relevant, please respond with "This query doesn't seem to be relevant to the source document."

"""


first_user_prompt = """
Context:
Always ensure that your tires are inflated to the correct pressure (page 6-2).
Under-inflated tires can cause poor handling and fuel economy (page 7-2).
The optimal tire pressures are stamped on the inside of the driver's side door (page 3-9).
Over-inflated tires can cause poor handling and uneven wear (page 1-4).

User Query:
How can I determine the correct tire pressure for my car?

"""

first_assistant = """
Answer:
Let's think step by step: The optimal tire pressure is mentioned in the context on page 3-9.
    Assuming optimal is correct, I have the answer about how the user can determine the correction tire pressure.
    The answer is: The optimal tire pressures are stamped on the inside of the driver's side door.
"""

second_user_prompt = """
Context:
Window wipers are required by law.
The wipers should be replaced every 6 months.
We recommend replacing the wipers every 6 months.
Wipers should be cleaned regularly.

User Query:
What brand of window wipers should I buy for my Mazda 3?

"""

second_asssistant = """
Answer:
Let's think step by step: The user is asking for a specific brand of window wipers. The context does not mention a specific brand.
"""

raqa_prompt = SystemRolePrompt(RAQA_PROMPT_TEMPLATE)
# raqa_prompt.add_message(first_user_prompt, first_assistant)
first_user = UserRolePrompt(first_user_prompt)
first_assistant = AssistantRolePrompt(first_assistant)
second_user = UserRolePrompt(second_user_prompt)
second_assistant = AssistantRolePrompt(second_asssistant)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}

Answer:
Let's think step by step:
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = raqa_prompt.create_message()
        formatted_first_user = first_user.create_message()
        formatted_first_assistant = first_assistant.create_message()
        formatted_second_user = second_user.create_message()
        formatted_second_assistant = second_assistant.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)
        
        return self.llm.run([formatted_system_prompt, formatted_first_user, formatted_first_assistant, formatted_second_user, formatted_second_assistant, formatted_user_prompt])

#### ❓QUESTION:

What changes could you make that would encourage the LLM to have a more well thought out and verbose response?

1. Provide few-shot Chain of Thought examples to encourage the model to reason it's answer out step by step
2. Ask it to first validate that the context is relevant to the query. Ask it to take a step by step approach to answering the question and make sure to provide reasoning and cite sources before actually sharing the final answer.


What is this method called? Few-Shot Chain of Thought reasoning

In [21]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [22]:
output = retrieval_augmented_qa_pipeline.run_pipeline("Should I care about tire pressure in my Mazda 3?")
print(output)

The user is asking whether they should care about tire pressure in their Mazda 3. The context indicates that maintaining the recommended tire pressure is crucial for the best ride, handling, and minimum tire wear. Additionally, the Tire Pressure Monitoring System (TPMS) is mentioned as a tool to help ensure the tire pressure is at the correct level. Based on this information, it is important to pay attention to and care about the tire pressure in your Mazda 3 to ensure optimal performance and safety.


### Visibility Tooling

This is great, but what if we wanted to add some visibility to our pipeline?

Let's use Weights and Biases as a visibility tool!

The first thing we'll need to do is create a Weights and Biases account and get an API key. 

You can follow the process outlined [here](https://docs.wandb.ai/quickstart) to do exactly that!

Now we can get the Weights and Biases dependency and add our key to our env. to begin!

In [17]:
!pip install -q -U wandb

In [23]:
# wandb_key = getpass("Weights and Biases API Key: ")
# os.environ["WANDB_API_KEY"] = wandb_key
from pathlib import Path
print(Path("RAQA_simon.ipynb").exists())

True


In [24]:
os.environ["WANDB_NOTEBOOK_NAME"] = "RAQA_simon.ipynb"
import wandb
# settings=wandb.Settings(disable_job_creation=True)
wandb.init(project="Visibility Example V7") #, settings=settings)

wandb: Currently logged in as: simonrmonk. Use `wandb login --relogin` to force relogin


Now we can integrate Weights and Biases into our `RetrievalAugmentedQAPipeline`.

```python
if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
```

The main things to consider here are how to populate the various fields to make sure we're tracking useful information. 

We'll use the `text_only` flag to ensure we can get detailed information about our LLM call!

You can check out all the parameters for Weights and Biases `Trace` [here](https://github.com/wandb/wandb/blob/653015a014281f45770aaf43627f64d9c4f04a32/wandb/sdk/data_types/trace_tree.py#L166)

In [25]:
import datetime
from wandb.sdk.data_types.trace_tree import Trace
inputs_dict = {}

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, wandb_project = None) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.wandb_project = wandb_project

    def run_pipeline(self, user_query: str) -> str:
        # global inputs_dict
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = raqa_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)
        formatted_first_user = first_user.create_message()
        formatted_first_assistant = first_assistant.create_message()
        formatted_second_user = second_user.create_message()
        formatted_second_assistant = second_assistant.create_message()
        
        start_time = datetime.datetime.now().timestamp() * 1000
        
        inputs = [formatted_system_prompt, formatted_first_user, formatted_first_assistant, formatted_second_user, formatted_second_assistant, formatted_user_prompt]

        try:
            openai_response = self.llm.run(inputs, text_only=False)
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "success"
            status_message = (None, )
            response_text = openai_response.choices[0].message.content
            token_usage = dict(openai_response.usage)
            model = openai_response.model

        except Exception as e:
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "error"
            status_message = str(e)
            response_text = ""
            token_usage = {}
            model = ""


        if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage,
                    "model_name" : model
                },
                # inputs= inputs_dict.copy(),
                inputs={"system_prompt" : formatted_system_prompt, "user_prompt_1": formatted_first_user, "assistant_1": formatted_first_assistant, "user_prompt_2": formatted_second_user, "assistant_2": formatted_second_assistant, "user_prompt_final" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
        
        return response_text if response_text else "We ran into an error. Please try again later. Full Error Message: " + status_message

In [26]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    wandb_project="LLM Visibility Example"
)

In [63]:
# retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?")

In [27]:
retrieval_augmented_qa_pipeline.run_pipeline("What should my tire pressure be?")

"The context provides detailed information about determining the recommended tire pressure for your vehicle:\n1. The tire label with inflation pressure is located on the driver's side B-pillar or on the edge of the driver's door frame.\n2. The recommended tire inflation pressure is listed in kPa and psi for the tires installed as original equipment on your vehicle.\n3. The recommended pressure is for cold tires after the vehicle has been parked for at least 3 hours.\n4. It is crucial to maintain the recommended pressure for the best ride, handling, and minimum tire wear.\n5. Check the tire pressure regularly to ensure it is at the proper inflation level.\n6. After adjusting the tire pressure, the tire pressure monitoring system initialization may be necessary for the system to operate normally.\n\nSo, to determine your tire pressure, refer to the tire label on the driver's side B-pillar or driver's door frame for the manufacturer's recommended cold tire inflation pressure."

In [28]:
wandb.finish()

Navigate to the Weights and Biases "run" link to see how your LLM is performing!

```
View run at YOUR LINK HERE
```

#### ❓QUESTION:

What is the `model_name` from the WandB `root_span` trace?

# Conclusion

In this notebook, we've gone through the steps required to create your own simple RAQA application!

Please feel free to extend this as much as you'd like. 